# STRIPS
### Autorzy: Jakub Kot, Dawid Małecki

## Cel ćwiczenia
Wykorzystanie algorytmu planowania przestrzeni stanów STRIPS. Algorytm ten jest wykorzystywany do planowania w dziedzinach, w których mamy do czynienia z przestrzenią stanów, w której mamy zdefiniowane akcje, które zmieniają stan świata.

## Opis problemu
Implementacja dziedziny problemu STRIPS, oraz kilku przykładowych problemów, oraz rozwiązanie ich.

## Realizacja rozwiązania
Zaimplementowano problem STIRPS w języku Python z pomocą biblioteki AIPython, a następnie przetestowano działanie.

## Podjęte próby rozwiązania
- Zaimplementowanie i rozwiązanie trywialnego problemu w celu nauki działania biblioteki
- Zaimplementowanie dziedziny problemu STRIPS
- Zdefiniowanie podstawowych akcji
- Zdefiniowanie stanów początkowych i końcowych
- Zdefiniowanie celu
- Zdefiniowanie problemu i podproblemów
- Dołączenie heurystyki

## Problem upieczenia ciasta

In [18]:
from stripsProblem import Planning_problem, STRIPS_domain, Strips
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS

# Cake

def eat():
    return "eat_cake"

def bake():
    return "bake_cake"

stmap = { 
        Strips(eat(), { "cake": True}, {"cake": False}),
        Strips(bake(), { "cake": False}, {"cake": True})
    }

# all possible actions
feature_domain_dict = { "cake": { True, False} }
strips_domain = STRIPS_domain(feature_domain_dict, stmap)
cake_problem = Planning_problem(strips_domain, { "cake": False}, { "cake": True})
SearcherMPP(Forward_STRIPS(cake_problem)).search()


Solution: {'cake': False}
   --bake_cake--> {'cake': True} (cost: 1)
 2 paths have been expanded and 0 paths remain in the frontier


{'cake': False}
   --bake_cake--> {'cake': True}

In [19]:
def searchForSolutions(searcher: SearcherMPP, iterations=1000):
    best_sol = None
    for i in range(iterations):
        print("Iteration: ", i)
        new_sol = searcher.search()
        if new_sol is None:
            break
        if best_sol is None or new_sol.cost < best_sol.cost:
            best_sol = new_sol
    return best_sol

## Walka ze smokiem
### Problemu
Rycerz musi pokonać smoka.

### Podproblemy:
- Rycerz musi zebrać wszystkie składniki (ziema i ogień)
- Rycerz musi zbierać składniki na zmianę, zaczynając od ognia
- Po zebraniu wszystkich składników musi z nich stworzyć kulę ognia
- Po pokoananiu smoka rycerz musi wrócić do zamku

### Heurystyka
Jeśli nie zebrano wszystkich składników - najkrótsza ścieżka do następnego składnika

Jeśli zebrano wszystkie składniki - najkrótsza ścieżka do smoka

Jeśli pokonano smoka - najkrótsza ścieżka do zamku

In [23]:
from stripsProblem import Planning_problem, STRIPS_domain, Strips
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS
from timeit import default_timer as timer

def move(x, y, nx, ny):
    return 'move_from_'+str(x)+str(y)+'_to_'+str(nx)+str(ny)

def attack(x, y):
    return "attack_dragon_at_" + str(x) + "_" + str(y)

def open():
    pass

def collectFire(x, y):
    return "collect_fire_at_" + str(x) + "_" + str(y)

def collectEarth(x, y):
    return "collect_earth_at_" + str(x) + "_" + str(y)

def buildFireball():
    return "build_fireball"

def get_fire(x, y):
    return f"fire_at_{x}_{y}"

def get_earth(x, y):
    return f"earth_at_{x}_{y}"

# nxn grid with player, castle, walls, and dragons
# End goal is kill the dragon and react the castle
# To kill the dragon, player has to be at dragon coordinates and has the fireball
# To build a fireball, the player needs k earth and l fire
# To attack, the player has to be at the same position as a dragon
# Player can open a chest but idk why - skip for now

def initWorld(n, dragon_coords, fire_coords, earth_coords):
    # nxn grid
    feature_domain_dict = { 
        "player": [(i, j) for i in range(n) for j in range(n)], 
        "dragon": (True, False),
        "fireball": (True, False), 
        "collectFire": (True, False),
    }

    for fire in fire_coords:
        feature_domain_dict[get_fire(*fire)] = (True, False)
    
    for earth in earth_coords:
        feature_domain_dict[get_earth(*earth)] = (True, False)

    # all possible player movements
    stmap = {
        Strips(move(x, y, x + dx, y + dy), { "player": (x, y)}, { "player": (x + dx, y + dy)}) 
            for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]
            for x in range(n) 
            for y in range(n)
            if x + dx in range(n) and y + dy in range(n)
    }

    # collecting fire
    stmap.update({
        Strips(collectFire(*coords), { "player": coords, get_fire(*coords): False, "collectFire": True }, { get_fire(*coords): True, "collectFire": False})
        for coords in fire_coords
    })

    # collecting earth
    stmap.update({
        Strips(collectEarth(*coords), { "player": coords, get_earth(*coords): False, "collectFire": False }, { get_earth(*coords): True, "collectFire": True})
        for coords in earth_coords
    })

    prereq_fireball = {
        "fireball": False,
    }

    for fire in fire_coords:
        prereq_fireball[get_fire(*fire)] = True
    
    for earth in earth_coords:
        prereq_fireball[get_earth(*earth)] = True

    # building a fireball
    stmap.update({
        Strips(buildFireball(), 
        prereq_fireball,
        { "fireball": True })
    })

    # attacking a dragon
    stmap.update({
       Strips(attack(dragon_coords[0], dragon_coords[1]), { "player": dragon_coords, "dragon": True, "fireball": True}, { "dragon": False, "fireball": False }) 
    })

    return STRIPS_domain(feature_domain_dict, stmap)


def searchForSolutions(searcher: SearcherMPP, iterations=1000):
    best_sol = None
    for i in range(iterations):
        print("Iteration: ", i)
        new_sol = searcher.search()
        if new_sol is None:
            break
        if best_sol is None or new_sol.cost < best_sol.cost:
            best_sol = new_sol
    return best_sol

def heur_func(state, goal):
    # calculate the heuristic based on the state
    player = state["player"]
    dragon = state["dragon_coords"]
    # get all the features related to fire locations
    firekeys = list(filter(lambda x: not state[x], filter(lambda x: "fire_at" in x, state.keys())))
    earthkeys = list(filter(lambda x: not state[x], filter(lambda x: "earth_at" in x, state.keys())))
    
    collectFire = state["collectFire"]
    fireball = state["fireball"]
    castle = goal["player"]

    def castle_dist():
        return abs(player[0] - castle[0]) + abs(player[1] - castle[1])
    
    def dragon_dist():
        return abs(player[0] - dragon[0]) + abs(player[1] - dragon[1])

    def fire_dist():
        firelocations = [f.split("_")[2:] for f in firekeys]
        return min([abs(player[0] - int(f[0])) + abs(player[1] - int(f[1])) for f in firelocations])
    
    def earth_dist():
        earthlocations = [f.split("_")[2:] for f in earthkeys]
        return min([abs(player[0] - int(f[0])) + abs(player[1] - int(f[1])) for f in earthlocations])

    # distance = manhattan distance
    # if dragon is dead, heuristic is the distance to the castle
    if not dragon:
        return castle_dist()
    # if dragon is alive and player has all the fire/earth, heuristic is distance to the dragon
    elif fireball or (len(firekeys) + len(earthkeys) == 0):
        return dragon_dist()
    # if dragon is alive, player has no fire/earth, and is collecting fire, heuristic is distance to the nearest fire
    elif collectFire:
        return fire_dist()
    # if dragon is alive, player has no fire/earth, and is collecting earth, heuristic is distance to the nearest earth
    else:
        return earth_dist()


def problemCreator(dragon, fire, earth, player_coords, castle, mapsize, iters=100, use_heuristic=True):
    world1 = initWorld(mapsize, dragon, fire, earth)
    start_state = {
        "player": player_coords,
        "dragon": True,
        "fireball": False,
        "collectFire": True,
        "dragon_coords": dragon,
    }

    for f in fire:
        start_state[get_fire(*f)] = False

    for e in earth:
        start_state[get_earth(*e)] = False

    goal_state = {
        "player": castle,
        "dragon": False
    }

    problem = Planning_problem(
        world1, 
        start_state, 
        goal_state
    )
    # sol = searchForSolutions(SearcherMPP(Forward_STRIPS(problem, heur=heur_func)), iters)
    start_time = timer()

    if use_heuristic:
        sol = SearcherMPP(Forward_STRIPS(problem, heur=heur_func)).search()
    else:
        sol = SearcherMPP(Forward_STRIPS(problem)).search()
    end_time = timer()

    print(sol)
    print(f"Cost of the solution: {sol.cost}")
    return sol, end_time - start_time

## Proste problemy

In [24]:
dragon = (1, 3)
fire = [(2, 2)]
earth = [(7, 5)]
player_coords = (1, 1)
castle = (4, 4)
_, time1 = problemCreator(dragon, fire, earth, player_coords, castle, 10, use_heuristic=True)
_, time2 = problemCreator(dragon, fire, earth, player_coords, castle, 10, use_heuristic=False)
print(f"Time taken for heuristic: {time1}")
print(f"Time taken without heuristic: {time2}")

Solution: {'player': (1, 1), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_2': False, 'earth_at_7_5': False}
   --move_from_11_to_12--> {'player': (1, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_2': False, 'earth_at_7_5': False}
   --move_from_12_to_22--> {'player': (2, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_2': False, 'earth_at_7_5': False}
   --collect_fire_at_2_2--> {'player': (2, 2), 'dragon': True, 'fireball': False, 'collectFire': False, 'dragon_coords': (1, 3), 'fire_at_2_2': True, 'earth_at_7_5': False}
   --move_from_22_to_32--> {'player': (3, 2), 'dragon': True, 'fireball': False, 'collectFire': False, 'dragon_coords': (1, 3), 'fire_at_2_2': True, 'earth_at_7_5': False}
   --move_from_32_to_33--> {'player': (3, 3), 'dragon': True, 'fireball': False, 'collectFire': False, 'dragon_coords': (1, 3), 'fire_at_2_2': True, 'earth_

In [25]:
dragon = (1, 3)
fire = [(2, 4), (8, 8)]
earth = [(7, 5)]
player_coords = (1, 1)
castle = (4, 4)
_, time1 = problemCreator(dragon, fire, earth, player_coords, castle, 10, use_heuristic=True)
_, time2 = problemCreator(dragon, fire, earth, player_coords, castle, 10, use_heuristic=False)
print(f"Time taken for heuristic: {time1}")
print(f"Time taken without heuristic: {time2}")

Solution: {'player': (1, 1), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_4': False, 'fire_at_8_8': False, 'earth_at_7_5': False}
   --move_from_11_to_12--> {'player': (1, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_4': False, 'fire_at_8_8': False, 'earth_at_7_5': False}
   --move_from_12_to_22--> {'player': (2, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_4': False, 'fire_at_8_8': False, 'earth_at_7_5': False}
   --move_from_22_to_23--> {'player': (2, 3), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_4': False, 'fire_at_8_8': False, 'earth_at_7_5': False}
   --move_from_23_to_24--> {'player': (2, 4), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_4': False, 'fire_at_8_8': False, 'earth_at_7_5': False}
   --collect_fire_at_2_4--> {'player': (2, 4), 

In [26]:
dragon = (0, 0)
fire = [(1, 1), (1, 2)]
earth = [(2, 1), (9, 9)]
player_coords = (2, 2)
castle = (9, 8)
_, time1 = problemCreator(dragon, fire, earth, player_coords, castle, 10, use_heuristic=True)
_, time2 = problemCreator(dragon, fire, earth, player_coords, castle, 10, use_heuristic=False)
print(f"Time taken for heuristic: {time1}")
print(f"Time taken without heuristic: {time2}")

Solution: {'player': (2, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (0, 0), 'fire_at_1_1': False, 'fire_at_1_2': False, 'earth_at_2_1': False, 'earth_at_9_9': False}
   --move_from_22_to_12--> {'player': (1, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (0, 0), 'fire_at_1_1': False, 'fire_at_1_2': False, 'earth_at_2_1': False, 'earth_at_9_9': False}
   --collect_fire_at_1_2--> {'player': (1, 2), 'dragon': True, 'fireball': False, 'collectFire': False, 'dragon_coords': (0, 0), 'fire_at_1_1': False, 'fire_at_1_2': True, 'earth_at_2_1': False, 'earth_at_9_9': False}
   --move_from_12_to_22--> {'player': (2, 2), 'dragon': True, 'fireball': False, 'collectFire': False, 'dragon_coords': (0, 0), 'fire_at_1_1': False, 'fire_at_1_2': True, 'earth_at_2_1': False, 'earth_at_9_9': False}
   --move_from_22_to_32--> {'player': (3, 2), 'dragon': True, 'fireball': False, 'collectFire': False, 'dragon_coords': (0, 0), 'fire_at_1_1': False

## Trudniejsze problemy 

In [27]:
dragon = (2, 11)
fire = [(2, 5), (5, 1), (10, 8), (1, 5)]
earth = [(4, 2), (0, 6), (3, 11)]
player_coords = (7, 2)
castle = (8, 8)
_, time1 = problemCreator(dragon, fire, earth, player_coords, castle, 12, use_heuristic=True)
_, time2 = problemCreator(dragon, fire, earth, player_coords, castle, 12, use_heuristic=False)
print(f"Time taken for heuristic: {time1}")
print(f"Time taken without heuristic: {time2}")

Solution: {'player': (7, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (2, 11), 'fire_at_2_5': False, 'fire_at_5_1': False, 'fire_at_10_8': False, 'fire_at_1_5': False, 'earth_at_4_2': False, 'earth_at_0_6': False, 'earth_at_3_11': False}
   --move_from_72_to_62--> {'player': (6, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (2, 11), 'fire_at_2_5': False, 'fire_at_5_1': False, 'fire_at_10_8': False, 'fire_at_1_5': False, 'earth_at_4_2': False, 'earth_at_0_6': False, 'earth_at_3_11': False}
   --move_from_62_to_52--> {'player': (5, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (2, 11), 'fire_at_2_5': False, 'fire_at_5_1': False, 'fire_at_10_8': False, 'fire_at_1_5': False, 'earth_at_4_2': False, 'earth_at_0_6': False, 'earth_at_3_11': False}
   --move_from_52_to_51--> {'player': (5, 1), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (2, 11), 'fire_at_2_5': False, 'fire_

In [28]:
dragon = (10, 11)
fire = [(2, 2), (10, 1), (1, 11), (1, 9), (0, 0)]
earth = [(7, 5), (6, 6), (9, 11), (7, 9)]
player_coords = (11, 10)
castle = (5, 5)
_, time1 = problemCreator(dragon, fire, earth, player_coords, castle, 12, use_heuristic=True)
_, time2 = problemCreator(dragon, fire, earth, player_coords, castle, 12, use_heuristic=False)
print(f"Time taken for heuristic: {time1}")
print(f"Time taken without heuristic: {time2}")

Solution: {'player': (11, 10), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (10, 11), 'fire_at_2_2': False, 'fire_at_10_1': False, 'fire_at_1_11': False, 'fire_at_1_9': False, 'fire_at_0_0': False, 'earth_at_7_5': False, 'earth_at_6_6': False, 'earth_at_9_11': False, 'earth_at_7_9': False}
   --move_from_1110_to_119--> {'player': (11, 9), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (10, 11), 'fire_at_2_2': False, 'fire_at_10_1': False, 'fire_at_1_11': False, 'fire_at_1_9': False, 'fire_at_0_0': False, 'earth_at_7_5': False, 'earth_at_6_6': False, 'earth_at_9_11': False, 'earth_at_7_9': False}
   --move_from_119_to_118--> {'player': (11, 8), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (10, 11), 'fire_at_2_2': False, 'fire_at_10_1': False, 'fire_at_1_11': False, 'fire_at_1_9': False, 'fire_at_0_0': False, 'earth_at_7_5': False, 'earth_at_6_6': False, 'earth_at_9_11': False, 'earth_at_7_9': False}
   --m

In [29]:
dragon = (10, 11)
fire = [(2, 2), (6, 1), (5, 9), (1, 9), (2, 8)]
earth = [(7, 5), (4, 6), (3, 11), (7, 9), (11, 11)]
player_coords = (11, 10)
castle = (5, 5)
_, time1 = problemCreator(dragon, fire, earth, player_coords, castle, 12, use_heuristic=True)
_, time2 = problemCreator(dragon, fire, earth, player_coords, castle, 12, use_heuristic=False)
print(f"Time taken for heuristic: {time1}")
print(f"Time taken without heuristic: {time2}")

Solution: {'player': (11, 10), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (10, 11), 'fire_at_2_2': False, 'fire_at_6_1': False, 'fire_at_5_9': False, 'fire_at_1_9': False, 'fire_at_2_8': False, 'earth_at_7_5': False, 'earth_at_4_6': False, 'earth_at_3_11': False, 'earth_at_7_9': False, 'earth_at_11_11': False}
   --move_from_1110_to_119--> {'player': (11, 9), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (10, 11), 'fire_at_2_2': False, 'fire_at_6_1': False, 'fire_at_5_9': False, 'fire_at_1_9': False, 'fire_at_2_8': False, 'earth_at_7_5': False, 'earth_at_4_6': False, 'earth_at_3_11': False, 'earth_at_7_9': False, 'earth_at_11_11': False}
   --move_from_119_to_118--> {'player': (11, 8), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (10, 11), 'fire_at_2_2': False, 'fire_at_6_1': False, 'fire_at_5_9': False, 'fire_at_1_9': False, 'fire_at_2_8': False, 'earth_at_7_5': False, 'earth_at_4_6': False, 'earth_at

## Przemyślenia i wnioski
- Heurystyka zazwyczaj nie przyspiesza działania (a przynajmniej nie w naszym przypadku)
- Algorytm stosunkowo szybko przeszukuje przestrzeń stanów, ale dla bardziej skomplikowanych problemów nadal może być to czasochłonne
- Implementacja dziedziny problemu STRIPS nie zawsze jest najprostsza. Wymaga zdefiniowania absolutnie wszystkich możliwych stanów i akcji.